# Network Anomaly Detection - Training on Google Colab

This notebook is set up for training the network anomaly detection model on Google Colab with GPU support.


## 1. Setup and Installation


In [ ]:
# Mount Google Drive (optional - if data is stored there)
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# Install dependencies
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install numpy pandas scikit-learn matplotlib seaborn tqdm huggingface-hub transformers tensorboard pyyaml wandb


In [ ]:
# Clone repository or upload files
# Option 1: If using git
# !git clone https://github.com/your-username/network_anomaly_detection.git
# %cd network_anomaly_detection

# Option 2: Upload files manually using Colab's file upload
# Then unzip if needed
# !unzip network_anomaly_detection.zip


## 2. Upload Data


In [ ]:
# Upload dataset
from google.colab import files
# files.upload()  # Uncomment to upload files

# Or copy from Drive
# !cp /content/drive/MyDrive/network_anomaly_detection/data/raw/embedded_system_network_security_dataset.csv ./data/raw/


## 3. Check GPU Availability


In [ ]:
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")


## 4. Configure Training


In [ ]:
# Update config for Colab if needed
import yaml

# Load config
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Update device settings for Colab
config['device']['use_cuda'] = torch.cuda.is_available()
config['device']['cuda_device'] = 0

# Enable augmented data usage
config['data']['use_augmented_data'] = True

# Optionally enable wandb logging (requires wandb login)
# config['logging']['use_wandb'] = True  # Uncomment to enable

# Save updated config
with open('config.yaml', 'w') as f:
    yaml.dump(config, f, default_flow_style=False)

print("Configuration updated for Colab")
print(f"CUDA enabled: {config['device']['use_cuda']}")
print(f"Using augmented data: {config['data']['use_augmented_data']}")
print(f"W&B logging: {config['logging']['use_wandb']}")


## 5. Data Augmentation


In [ ]:
# Run data augmentation
# This will create augmented training/validation data and hold out test data
print("Running data augmentation...")
!python augment_data.py --config config.yaml --noise-scale 0.1 --target-positive-ratio 0.3 --num-augmentations 1

print("\nAugmentation complete! Check data/augmented/ directory for:")
print("  - train_data_augmented.csv")
print("  - val_data.csv")
print("  - test_data.csv")


In [ ]:
## 6. K-Fold Cross Validation Training


# Train with K-fold cross validation
# This will create models for each fold in checkpoints/fold_1/, fold_2/, etc.
print("Starting K-fold cross validation...")
!python train.py --config config.yaml --use-cv

print("\nK-fold CV complete! Check checkpoints/ directory for fold models.")


In [ ]:
## 7. Select Best Model and Train Final Model


# Train final model using best weights from k-fold CV
# This script automatically:
# 1. Finds all fold checkpoints
# 2. Selects the best fold (lowest validation loss)
# 3. Initializes model with those weights
# 4. Trains on full training data (train + val combined)
# 5. Saves to checkpoints/final_model/best_model.pt

print("Training final model from best CV fold...")
!python train_final_model.py --config config.yaml

print("\nFinal model training complete! Model saved to checkpoints/final_model/best_model.pt")


In [ ]:
## 8. Test on Each K-Fold Model


In [ ]:
# Test each fold model on the test set
import os
import glob
import subprocess

checkpoint_dir = "checkpoints"
fold_pattern = os.path.join(checkpoint_dir, "fold_*", "best_model.pt")
fold_checkpoints = glob.glob(fold_pattern)

print(f"Found {len(fold_checkpoints)} fold checkpoints to test\n")

fold_results = []
for checkpoint_path in sorted(fold_checkpoints):
    fold_num = checkpoint_path.split('/')[-2].split('_')[1]
    print(f"{'='*60}")
    print(f"Testing Fold {fold_num}")
    print(f"{'='*60}")
    print(f"Checkpoint: {checkpoint_path}\n")
    
    # Test this fold model
    result = subprocess.run(
        ['python', 'test.py', '--config', 'config.yaml', '--checkpoint', checkpoint_path],
        capture_output=True,
        text=True
    )
    print(result.stdout)
    if result.stderr:
        print("Errors:", result.stderr)
    
    print(f"\nFold {fold_num} testing complete!\n")

print(f"\n{'='*60}")
print("All fold models tested!")
print(f"{'='*60}")


## 9. Test Final Model


In [ ]:
# Test the final model
print("Testing final model...")
!python test.py --config config.yaml --final-model

print("\nFinal model testing complete!")


## 10. Monitor Training with TensorBoard


In [ ]:
# Load TensorBoard extension
%load_ext tensorboard

# Start TensorBoard to view all training logs
# This will show:
# - K-fold CV training logs (runs/cv_fold_1_*, cv_fold_2_*, etc.)
# - Final model training logs (runs/final_model_*)
# - Test evaluation logs (runs/test_*)
%tensorboard --logdir runs --port 6006


## 11. Optional: Upload to Hugging Face


In [ ]:
# Login to Hugging Face (optional)
from huggingface_hub import login
# login()  # Uncomment and run to login (will prompt for token)


In [ ]:
# Upload final model to Hugging Face (optional)
# !python upload_to_hub.py --checkpoint checkpoints/final_model/best_model.pt --repo-id your-username/network-anomaly-detector


## 12. Download Results (Optional)


In [ ]:
# Download checkpoints and results to Google Drive (optional)
# !cp -r checkpoints /content/drive/MyDrive/network_anomaly_detection/
# !cp -r runs /content/drive/MyDrive/network_anomaly_detection/
# !cp -r data/augmented /content/drive/MyDrive/network_anomaly_detection/data/
